In [12]:
import pandas as pd
import json
import re

In [13]:
# Read both sheets of the Excel file
df_duplicated = pd.read_excel("chats.298.xlsx", sheet_name="Input")
df_numbers = pd.read_excel("chats.298.xlsx", sheet_name="Output")


In [14]:
with open("city.json", "r", encoding="utf-8") as json_file:
    target_ids = json.load(json_file)


In [15]:
city_names_ar = [city["label"] for city in target_ids]
city_names_en = [city["value"] for city in target_ids]
city_patterns_ar = re.compile(
    '|'.join(map(re.escape, city_names_ar)), re.IGNORECASE)
city_patterns_en = re.compile(
    '|'.join(map(re.escape, city_names_en)), re.IGNORECASE)


In [16]:
def extract_cities(chat_content):
    matched_cities_ar = city_patterns_ar.findall(chat_content)
    matched_cities_en = city_patterns_en.findall(chat_content)
    chat_cities = matched_cities_ar + matched_cities_en
    return chat_cities

In [17]:
extracted_cities_dict = {}


In [18]:
for number in df_duplicated['mobile2']:
    if number not in extracted_cities_dict:
        extracted_cities = []

        # Iterate through chat contents associated with the current phone number
        # Iterate through chat contents in the 'content' column of 'df_duplicated'
        # that are associated with the current phone number.
        for chat_content in df_duplicated[df_duplicated['mobile2'] == number]['content']:
            if isinstance(chat_content, str):
                extracted_cities.extend(extract_cities(chat_content))
        extracted_cities_dict[number] = extracted_cities


In [19]:
new_data = []


In [20]:
for number in df_numbers['Mobile2']:
    if number not in extracted_cities_dict:
        new_data.append([number, ''])
    else:
        cities = extracted_cities_dict[number]
        new_data.append([number, ', '.join(cities)])



In [21]:
new_df = pd.DataFrame(new_data, columns=['Mobile2', 'City'])

In [25]:
merged_df = pd.merge(df_numbers, new_df, how='left', left_on='Mobile2', right_on='Mobile2')


In [26]:
with pd.ExcelWriter("chats.298.xlsx", engine='openpyxl') as writer:
    merged_df.to_excel(writer, sheet_name="Output", index=False)
